# *Live Feed Hard Coded SpliceVid

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import math
import os

# Create RepVids directory in "Deep Dive AI Summer 2025" folder
save_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepVids")
os.makedirs(save_dir, exist_ok=True)

# Initialize variables
frame_buffer = []
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
buffer_size = 90  # Number of frames to keep in buffer

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

def calc_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else angle

cap = cv2.VideoCapture(0)
counter = 0
stage = None

with mp_pose.Pose(min_detection_confidence=0.5,
                  min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Always add current frame to buffer
        frame_buffer.append(image.copy())
        """
        if len(frame_buffer) > buffer_size:
            frame_buffer.pop(0)  # Keep only last 90 frames
        """

        label = "None"

        try:
            landmarks = results.pose_landmarks.landmark
            h, w, _ = image.shape

            # Get right side landmarks
            shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

            # Visibility check
            visible = all(landmarks[i].visibility > 0.7 for i in [
                mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
                mp_pose.PoseLandmark.RIGHT_ELBOW.value,
                mp_pose.PoseLandmark.RIGHT_WRIST.value,
                mp_pose.PoseLandmark.RIGHT_HIP.value,
                mp_pose.PoseLandmark.RIGHT_KNEE.value,
                mp_pose.PoseLandmark.RIGHT_ANKLE.value
            ])
            
            # Horizontal filter
            locs = np.array([
                shoulder[1] * h, elbow[1] * h, wrist[1] * h,
                hip[1] * h, knee[1] * h, ankle[1] * h
            ])
            data_range = np.max(locs) - np.min(locs)
            if data_range > 0.5 * h:
                visible = False
            downUp = False
            

            if visible:
                elbow_angle = calc_angle(shoulder, elbow, wrist)
                back_angle = calc_angle(shoulder, hip, knee)
                knee_angle = calc_angle(hip, knee, ankle)

                # Pushup logic
                percent = np.interp(elbow_angle, [90, 160], [0, 100])
                
                # Store previous counter to detect changes
                prev_counter = counter

                # Track full cycle: up -> down -> up
                if elbow_angle > 165:
                    if stage == "down":
                        stage = "up"
                        counter += 1
                    else:
                        stage = "up"
                elif elbow_angle < 95 and stage == "up":
                    stage = "down"

                # SAVE VIDEO EVERY TIME COUNTER INCREASES
                if counter > prev_counter:
                    filename = os.path.join(save_dir, f'rep_{counter:03d}.mp4')
                    
                    if len(frame_buffer) > 0:
                        out = cv2.VideoWriter(filename, fourcc, 20.0, (image.shape[1], image.shape[0]))
                        
                        for f in frame_buffer:
                            out.write(f)
                        out.release()
                        
                        print(f"Saved rep {counter} to: {filename} ({len(frame_buffer)} frames)")

                        frame_buffer = []  # Reset buffer for next rep
                    else:
                        print(f"No frames to save for rep {counter}")

                label = "Pushup"

                # Draw angle + percent
                cv2.putText(image, f'Elbow: {int(elbow_angle)}', (10, 100),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                cv2.putText(image, f'{int(percent)}%', (10, 140),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                label = "None"

        except:
            pass

        # Draw buffer status
        cv2.putText(image, f'Buffer: {len(frame_buffer)}/{buffer_size}', (10, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)

        # Draw pose
        mp_drawing.draw_landmarks(
            image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=4),
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2))

        # Label
        cv2.putText(image, f'Form: {label}', (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 2)

        # Rep counter
        cv2.rectangle(image, (image.shape[1] - 160, 0), (image.shape[1], 80), (0, 0, 0), -1)
        cv2.putText(image, 'REPS', (image.shape[1] - 150, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        cv2.putText(image, str(counter), (image.shape[1] - 140, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 4)

        cv2.imshow('Push-Up Detector', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Saved rep 1 to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepVids\rep_001.mp4 (539 frames)
Saved rep 2 to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepVids\rep_002.mp4 (41 frames)
Saved rep 3 to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepVids\rep_003.mp4 (39 frames)


# Run3 Code

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import math
import os
import zipfile
import tempfile
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd

# Create directories
save_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepVids")
failed_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepFailed")
os.makedirs(save_dir, exist_ok=True)
os.makedirs(failed_dir, exist_ok=True)

# Initialize variables
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
buffer_size = 90

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Track results for confusion matrix
results_data = []

def calc_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else angle

def process_video_file(video_file_path, output_prefix):
    frame_buffer = []
    all_frames = []  # Store ALL frames for potential failed video save
    
    cap = cv2.VideoCapture(video_file_path)
    counter = 0
    stage = None
    reps_found = 0
    
    print(f"🎬 Processing: {video_file_path}")
    
    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print(f"✅ Finished processing {video_file_path}")
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Add frames to both buffers
            frame_buffer.append(image.copy())
            all_frames.append(image.copy())

            label = "None"

            # 🔥 TRY BOTH RIGHT AND LEFT SIDE LANDMARKS
            try:
                landmarks = results.pose_landmarks.landmark
                h, w, _ = image.shape

                # Check RIGHT side landmarks
                right_landmarks = [
                    mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
                    mp_pose.PoseLandmark.RIGHT_ELBOW.value,
                    mp_pose.PoseLandmark.RIGHT_WRIST.value,
                    mp_pose.PoseLandmark.RIGHT_HIP.value,
                    mp_pose.PoseLandmark.RIGHT_KNEE.value,
                ]
                
                # Check LEFT side landmarks  
                left_landmarks = [
                    mp_pose.PoseLandmark.LEFT_SHOULDER.value,
                    mp_pose.PoseLandmark.LEFT_ELBOW.value,
                    mp_pose.PoseLandmark.LEFT_WRIST.value,
                    mp_pose.PoseLandmark.LEFT_HIP.value,
                    mp_pose.PoseLandmark.LEFT_KNEE.value,
                ]

                # Calculate visibility scores for both sides
                right_visibility = sum(landmarks[i].visibility for i in right_landmarks) / len(right_landmarks)
                left_visibility = sum(landmarks[i].visibility for i in left_landmarks) / len(left_landmarks)

                # Use the side with better visibility (threshold 0.7)
                use_right_side = right_visibility >= 0.7
                use_left_side = left_visibility >= 0.7

                # Prefer right side if both are good, otherwise use the better one
                if use_right_side and (right_visibility >= left_visibility or not use_left_side):
                    # Use RIGHT side
                    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                    
                    visible = right_visibility >= 0.7
                    side_used = "RIGHT"
                    
                elif use_left_side:
                    # Use LEFT side
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                    
                    visible = left_visibility >= 0.7
                    side_used = "LEFT"
                    
                else:
                    # Neither side meets visibility threshold
                    visible = False
                    side_used = "NONE"

                # Optional: Add debug info
                # print(f"Right vis: {right_visibility:.2f}, Left vis: {left_visibility:.2f}, Using: {side_used}")

                # Continue with existing horizontal filter if visible
                if visible:
                    # Horizontal filter
                    locs = np.array([
                        shoulder[1] * h, elbow[1] * h, wrist[1] * h,
                        hip[1] * h, knee[1] * h, ankle[1] * h
                    ])
                    data_range = np.max(locs) - np.min(locs)
                    # if data_range > 0.7 * h:
                    #     visible = False

                    if visible:
                        elbow_angle = calc_angle(shoulder, elbow, wrist)
                        
                        # Store previous counter to detect changes
                        prev_counter = counter

                        # Track full cycle: up -> down -> up
                        if elbow_angle > 130:
                            if stage == "down":
                                stage = "up"
                                counter += 1
                            else:
                                stage = "up"
                        elif elbow_angle < 110 and stage == "up":
                            stage = "down"

                        # SAVE VIDEO EVERY TIME COUNTER INCREASES
                        if counter > prev_counter:
                            reps_found += 1
                            # Create unique filename with video name
                            video_name = os.path.splitext(os.path.basename(video_file_path))[0]
                            filename = os.path.join(save_dir, f'{output_prefix}_{video_name}_rep_{counter:03d}.mp4')
                            
                            if len(frame_buffer) > 0:
                                out = cv2.VideoWriter(filename, fourcc, 20.0, (image.shape[1], image.shape[0]))
                                
                                for f in frame_buffer:
                                    out.write(f)
                                out.release()
                                
                                print(f"✅ Saved {output_prefix} rep {counter} ({side_used}) to: {filename} ({len(frame_buffer)} frames)")
                                frame_buffer = []  # Reset buffer for next rep
                            else:
                                print(f"❌ No frames to save for rep {counter}")

                        label = "Pushup"

            except Exception as e:
                print(f"❌ Error processing frame: {e}")
                pass

    cap.release()
    
    # 🚨 CHECK IF NO REPS WERE FOUND - SAVE FULL VIDEO TO FAILED FOLDER
    if reps_found == 0:
        video_name = os.path.splitext(os.path.basename(video_file_path))[0]
        failed_filename = os.path.join(failed_dir, f'{output_prefix}_{video_name}_FAILED.mp4')
        
        if len(all_frames) > 0:
            out = cv2.VideoWriter(failed_filename, fourcc, 20.0, (all_frames[0].shape[1], all_frames[0].shape[0]))
            
            for frame in all_frames:
                out.write(frame)
            out.release()
            
            print(f"🚨 NO REPS FOUND - Saved full video to: {failed_filename}")
            
            # Record result for confusion matrix
            results_data.append({
                'true_label': output_prefix,
                'predicted_label': 'failed',
                'video_name': video_name,
                'output_folder': 'RepFailed'
            })
        else:
            print(f"❌ No frames to save for failed video: {video_file_path}")
            results_data.append({
                'true_label': output_prefix,
                'predicted_label': 'failed',
                'video_name': video_name,
                'output_folder': 'RepFailed'
            })
    else:
        # Record successful processing
        video_name = os.path.splitext(os.path.basename(video_file_path))[0]
        results_data.append({
            'true_label': output_prefix,
            'predicted_label': 'success',
            'video_name': video_name,
            'output_folder': 'RepVids',
            'reps_found': reps_found
        })
        print(f"✅ Successfully found {reps_found} reps in {video_name}")
    
    print(f"🏁 Completed processing: {video_file_path}\n")

# Extract ZIP file
zip_path = r"C:\Users\12484\Downloads\fitDataa.zip"
temp_dir = tempfile.mkdtemp()

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Define paths to video directories
correct_videos_path = os.path.join(temp_dir, "fitDataa", "Correct sequence")
incorrect_videos_path = os.path.join(temp_dir, "fitDataa", "Wrong sequence")

# 🎯 PROCESS ALL VIDEOS IN BOTH DIRECTORIES
print("🎬 Processing CORRECT videos:")
for video_file in os.listdir(correct_videos_path):
    if video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        video_path = os.path.join(correct_videos_path, video_file)
        process_video_file(video_path, "correct")

print("\n🎬 Processing INCORRECT videos:")
for video_file in os.listdir(incorrect_videos_path):
    if video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        video_path = os.path.join(incorrect_videos_path, video_file)
        process_video_file(video_path, "incorrect")

print(f"\n🎬 All processed videos saved to: {save_dir}")
print(f"🚨 Failed videos saved to: {failed_dir}")


# 📊 CREATE Dataframe
df_results = pd.DataFrame(results_data)


🎬 Processing CORRECT videos:
🎬 Processing: C:\Users\12484\AppData\Local\Temp\tmp8j7l1e_d\fitDataa\Correct sequence\Copy of push up 1.mp4
✅ Finished processing C:\Users\12484\AppData\Local\Temp\tmp8j7l1e_d\fitDataa\Correct sequence\Copy of push up 1.mp4
🚨 NO REPS FOUND - Saved full video to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepFailed\correct_Copy of push up 1_FAILED.mp4
🏁 Completed processing: C:\Users\12484\AppData\Local\Temp\tmp8j7l1e_d\fitDataa\Correct sequence\Copy of push up 1.mp4

🎬 Processing: C:\Users\12484\AppData\Local\Temp\tmp8j7l1e_d\fitDataa\Correct sequence\Copy of push up 100.mp4
✅ Saved correct rep 1 (RIGHT) to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepVids\correct_Copy of push up 100_rep_001.mp4 (92 frames)
✅ Finished processing C:\Users\12484\AppData\Local\Temp\tmp8j7l1e_d\fitDataa\Correct sequence\Copy of push up 100.mp4
✅ Successfully found 1 reps in Copy of push up 100
🏁 Completed processing: C:\Users\12484\AppData\Local\Temp\tmp8j7l1e_d\fit

# *Run4 Code 

In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import math
import os
import zipfile
import tempfile
import shutil
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import pandas as pd

# Create directories
save_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepVids")
failed_dir = os.path.join(os.path.expanduser("~"), "Desktop", "Deep Dive AI Summer 2025", "RepFailed")
os.makedirs(save_dir, exist_ok=True)
os.makedirs(failed_dir, exist_ok=True)

# Initialize variables
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
buffer_size = 90

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Track results for confusion matrix
results_data = []

def calc_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians * 180.0 / np.pi)
    return 360 - angle if angle > 180 else angle

def process_video_file(video_file_path, output_prefix):
    frame_buffer = []
    all_frames = []  # Store ALL frames for potential failed video save
    
    cap = cv2.VideoCapture(video_file_path)
    counter = 0
    stage = None
    reps_found = 0
    
    print(f"Processing: {video_file_path}")
    
    with mp_pose.Pose(min_detection_confidence=0.5,
                      min_tracking_confidence=0.5) as pose:

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print(f"Finished processing {video_file_path}")
                break

            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Add frames to both buffers
            frame_buffer.append(image.copy())
            all_frames.append(image.copy())

            label = "None"

            # TRY BOTH RIGHT AND LEFT SIDE LANDMARKS
            try:
                landmarks = results.pose_landmarks.landmark
                h, w, _ = image.shape

                # Check RIGHT side landmarks
                right_landmarks = [
                    mp_pose.PoseLandmark.RIGHT_SHOULDER.value,
                    mp_pose.PoseLandmark.RIGHT_ELBOW.value,
                    mp_pose.PoseLandmark.RIGHT_WRIST.value,
                    mp_pose.PoseLandmark.RIGHT_HIP.value,
                    mp_pose.PoseLandmark.RIGHT_KNEE.value,
                    mp_pose.PoseLandmark.RIGHT_ANKLE.value
                ]
                
                # Check LEFT side landmarks  
                left_landmarks = [
                    mp_pose.PoseLandmark.LEFT_SHOULDER.value,
                    mp_pose.PoseLandmark.LEFT_ELBOW.value,
                    mp_pose.PoseLandmark.LEFT_WRIST.value,
                    mp_pose.PoseLandmark.LEFT_HIP.value,
                    mp_pose.PoseLandmark.LEFT_KNEE.value,
                    mp_pose.PoseLandmark.LEFT_ANKLE.value
                ]

                # Calculate visibility scores for both sides
                right_visibility = sum(landmarks[i].visibility for i in right_landmarks) / len(right_landmarks)
                left_visibility = sum(landmarks[i].visibility for i in left_landmarks) / len(left_landmarks)

                # Use the side with better visibility (threshold 0.7)
                use_right_side = right_visibility >= 0.7
                use_left_side = left_visibility >= 0.7

                # Prefer right side if both are good, otherwise use the better one
                if use_right_side and (right_visibility >= left_visibility or not use_left_side):
                    # Use RIGHT side
                    shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                    
                    visible = right_visibility >= 0.7
                    side_used = "RIGHT"
                    
                elif use_left_side:
                    # Use LEFT side
                    shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                    
                    visible = left_visibility >= 0.7
                    side_used = "LEFT"
                    
                else:
                    # Neither side meets visibility threshold
                    visible = False
                    side_used = "NONE"

                # Optional: Add debug info
                # print(f"Right vis: {right_visibility:.2f}, Left vis: {left_visibility:.2f}, Using: {side_used}")

                # Continue with existing horizontal filter if visible
                if visible:
                    # Horizontal filter
                    locs = np.array([
                        shoulder[1] * h, elbow[1] * h, wrist[1] * h,
                        hip[1] * h, knee[1] * h, ankle[1] * h
                    ])
                    data_range = np.max(locs) - np.min(locs)
                    # if data_range > 0.5 * h:
                    #     visible = False

                    if visible:
                        elbow_angle = calc_angle(shoulder, elbow, wrist)
                        
                        # Store previous counter to detect changes
                        prev_counter = counter

                        # Track full cycle: up -> down -> up
                        if elbow_angle > 155:
                            if stage == "down":
                                stage = "up"
                                counter += 1
                            else:
                                stage = "up"
                        elif elbow_angle < 110 and stage == "up":
                            stage = "down"

                        # SAVE VIDEO EVERY TIME COUNTER INCREASES
                        if counter > prev_counter:
                            reps_found += 1
                            # Create unique filename with video name
                            video_name = os.path.splitext(os.path.basename(video_file_path))[0]
                            filename = os.path.join(save_dir, f'{output_prefix}_{video_name}_rep_{counter:03d}.mp4')
                            
                            if len(frame_buffer) > 0:
                                out = cv2.VideoWriter(filename, fourcc, 20.0, (image.shape[1], image.shape[0]))
                                
                                for f in frame_buffer:
                                    out.write(f)
                                out.release()
                                
                                print(f"Saved {output_prefix} rep {counter} ({side_used}) to: {filename} ({len(frame_buffer)} frames)")
                                frame_buffer = []  # Reset buffer for next rep
                            else:
                                print(f"No frames to save for rep {counter}")

                        label = "Pushup"

            except Exception as e:
                print(f"Error processing frame: {e}")
                pass

    cap.release()
    
    # CHECK IF NO REPS WERE FOUND - SAVE FULL VIDEO TO FAILED FOLDER
    if reps_found == 0:
        video_name = os.path.splitext(os.path.basename(video_file_path))[0]
        failed_filename = os.path.join(failed_dir, f'{output_prefix}_{video_name}_FAILED.mp4')
        
        if len(all_frames) > 0:
            out = cv2.VideoWriter(failed_filename, fourcc, 20.0, (all_frames[0].shape[1], all_frames[0].shape[0]))
            
            for frame in all_frames:
                out.write(frame)
            out.release()
            
            print(f"NO REPS FOUND - Saved full video to: {failed_filename}")
            
            # Record result for confusion matrix
            results_data.append({
                'true_label': output_prefix,
                'predicted_label': 'failed',
                'video_name': video_name,
                'output_folder': 'RepFailed'
            })
        else:
            print(f"No frames to save for failed video: {video_file_path}")
            results_data.append({
                'true_label': output_prefix,
                'predicted_label': 'failed',
                'video_name': video_name,
                'output_folder': 'RepFailed'
            })
    else:
        # Record successful processing
        video_name = os.path.splitext(os.path.basename(video_file_path))[0]
        results_data.append({
            'true_label': output_prefix,
            'predicted_label': 'success',
            'video_name': video_name,
            'output_folder': 'RepVids',
            'reps_found': reps_found
        })
        print(f"Successfully found {reps_found} reps in {video_name}")
    
    print(f"Completed processing: {video_file_path}\n")

# Extract ZIP file
zip_path = r"C:\Users\12484\Downloads\fitDataa.zip"
temp_dir = tempfile.mkdtemp()

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

# Define paths to video directories
correct_videos_path = os.path.join(temp_dir, "fitDataa", "Correct sequence")
incorrect_videos_path = os.path.join(temp_dir, "fitDataa", "Wrong sequence")

# PROCESS ALL VIDEOS IN BOTH DIRECTORIES
print("Processing CORRECT videos:")
for video_file in os.listdir(correct_videos_path):
    if video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        video_path = os.path.join(correct_videos_path, video_file)
        process_video_file(video_path, "correct")

print("\nProcessing INCORRECT videos:")
for video_file in os.listdir(incorrect_videos_path):
    if video_file.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        video_path = os.path.join(incorrect_videos_path, video_file)
        process_video_file(video_path, "incorrect")

print(f"\nAll processed videos saved to: {save_dir}")
print(f"Failed videos saved to: {failed_dir}")


# CREATE Dataframe
df_results = pd.DataFrame(results_data)


Processing CORRECT videos:
Processing: C:\Users\12484\AppData\Local\Temp\tmpq90x1f6l\fitDataa\Correct sequence\Copy of push up 1.mp4
Finished processing C:\Users\12484\AppData\Local\Temp\tmpq90x1f6l\fitDataa\Correct sequence\Copy of push up 1.mp4
NO REPS FOUND - Saved full video to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepFailed\correct_Copy of push up 1_FAILED.mp4
Completed processing: C:\Users\12484\AppData\Local\Temp\tmpq90x1f6l\fitDataa\Correct sequence\Copy of push up 1.mp4

Processing: C:\Users\12484\AppData\Local\Temp\tmpq90x1f6l\fitDataa\Correct sequence\Copy of push up 100.mp4
Saved correct rep 1 (RIGHT) to: C:\Users\12484\Desktop\Deep Dive AI Summer 2025\RepVids\correct_Copy of push up 100_rep_001.mp4 (99 frames)
Finished processing C:\Users\12484\AppData\Local\Temp\tmpq90x1f6l\fitDataa\Correct sequence\Copy of push up 100.mp4
Successfully found 1 reps in Copy of push up 100
Completed processing: C:\Users\12484\AppData\Local\Temp\tmpq90x1f6l\fitDataa\Correct sequen

In [ ]:

                if visible:
                    elbow_angle = calc_angle(shoulder, elbow, wrist)
                    hip_angle = calc_angle(shoulder, hip, knee)
                    
                    # Store previous counter to detect changes
                    prev_counter = counter
                    
                    # DUAL REP DETECTION: ELBOW OR HIP BASED

                    # METHOD 1: Elbow-based detection (original)
                    elbow_rep_detected = False
                    if elbow_angle > 150:
                        if stage == "down":
                            stage = "up"
                            counter += 1
                            elbow_rep_detected = True
                        else:
                            stage = "up"
                    elif elbow_angle < 110 and stage == "up":
                        stage = "down"
                    
                    # METHOD 2: Hip-based detection (NEW - straight to bent outward)
                    hip_rep_detected = False
                    if not elbow_rep_detected:  # Only check hip if elbow didn't detect a rep
                        # Detect hip going from straight (high angle ~170°+) to bent outward (lower angle ~120°-)
                        if hip_angle < 120:  # Hip bent outward (bottom of pushup)
                            if stage == "hip_up":
                                stage = "hip_down"
                        elif hip_angle > 160:  # Hip straight (top of pushup)
                            if stage == "hip_down":
                                stage = "hip_up"
                                counter += 1
                                hip_rep_detected = True
                            else:
                                stage = "hip_up"
                    

                    # SAVE VIDEO EVERY TIME COUNTER INCREASES (from either method)
                    if counter > prev_counter:
                        reps_found += 1
                        # Create unique filename with video name
                        video_name = os.path.splitext(os.path.basename(video_file_path))[0]
                        # ADD DETECTION METHOD TO FILENAME
                        detection_method = "ELBOW" if elbow_rep_detected else "HIP" if hip_rep_detected else "UNKNOWN"
                        filename = os.path.join(save_dir, f'{output_prefix}_{video_name}_rep_{counter:03d}_{detection_method}.mp4')
                        
                        if len(frame_buffer) > 0:
                            out = cv2.VideoWriter(filename, fourcc, 20.0, (image.shape[1], image.shape[0]))
                            
                            for f in frame_buffer:
                                out.write(f)
                            out.release()
                            
                            print(f"Saved {output_prefix} rep {counter} ({side_used}) to: {filename} ({len(frame_buffer)} frames)")
                            frame_buffer = []  # Reset buffer for next rep
                        else:
                            print(f"No frames to save for rep {counter}")

                    label = "Pushup"

            except Exception as e:
                print(f"Error processing frame: {e}")
                pass

In [2]:
df_results

,true_label,predicted_label,video_name,output_folder,reps_found
0,correct,failed,Copy of push up 1,RepFailed,NaN
1,correct,success,Copy of push up 100,RepVids,1.0
2,correct,success,Copy of push up 100_flipped,RepVids,1.0
3,correct,success,Copy of push up 101,RepVids,1.0
4,correct,success,Copy of push up 101_flipped,RepVids,1.0
...,...,...,...,...,...
195,incorrect,success,Copy of push up 57_flipped,RepVids,1.0
196,incorrect,success,Copy of push up 58,RepVids,1.0
197,incorrect,success,Copy of push up 58_flipped,RepVids,1.0
198,incorrect,failed,Copy of push up 81,RepFailed,NaN


In [3]:
df_results.head()

print(f"code failed {len(df_results[df_results['predicted_label'] == 'failed'])}")
print(f"code success {len(df_results[df_results['predicted_label'] == 'success'])}")

print(f"\nlabelled correct: {len(df_results[df_results['true_label'] == 'correct'])}")
print(f"code failed {len(df_results[(df_results['predicted_label'] == 'failed') & (df_results['true_label'] == 'correct')])}")
print(f"code success {len(df_results[(df_results['predicted_label'] == 'success') & (df_results['true_label'] == 'correct')])}")

print(f"\nlabelled incorrect: {len(df_results[df_results['true_label'] == 'incorrect'])}")
print(f"code failed {len(df_results[(df_results['predicted_label'] == 'failed') & (df_results['true_label'] == 'incorrect')])}")
print(f"code success {len(df_results[(df_results['predicted_label'] == 'success') & (df_results['true_label'] == 'incorrect')])}")

code failed 21
code success 179

labelled correct: 100
code failed 8
code success 92

labelled incorrect: 100
code failed 13
code success 87


In [5]:
print(f"run1: \ncode failed {145}\ncode success {55}\n\nlabelled correct: {100}\ncode failed {73}\ncode success {27}\n\nlabelled incorrect: {100}\ncode failed {72}\ncode success {28}")
print(f"\nrun2: \ncode failed {107}\ncode success {93}\n\nlabelled correct: {100}\ncode failed {52}\ncode success {48}\n\nlabelled incorrect: {100}\ncode failed {55}\ncode success {45}")
print(f"\nrun3: \ncode failed {20}\ncode success {180}\n\nlabelled correct: {100}\ncode failed {7}\ncode success {93}\n\nlabelled incorrect: {100}\ncode failed {13}\ncode success {87}")
print(f"\nrun4: \ncode failed {19}\ncode success {179}\n\nlabelled correct: {100}\ncode failed {8}\ncode success {92}\n\nlabelled incorrect: {100}\ncode failed {13}\ncode success {87}")


run1: 
code failed 145
code success 55

labelled correct: 100
code failed 73
code success 27

labelled incorrect: 100
code failed 72
code success 28

run2: 
code failed 107
code success 93

labelled correct: 100
code failed 52
code success 48

labelled incorrect: 100
code failed 55
code success 45

run3: 
code failed 20
code success 180

labelled correct: 100
code failed 7
code success 93

labelled incorrect: 100
code failed 13
code success 87

run4: 
code failed 19
code success 179

labelled correct: 100
code failed 8
code success 92

labelled incorrect: 100
code failed 13
code success 87


In [10]:
print(f"run1: \ncode failed {145}\ncode success {55}\n\nlabelled correct: {100}\ncode failed {73}\ncode success {27}\n\nlabelled incorrect: {100}\ncode failed {72}\ncode success {28}")
"""
if data_range > 0.5 * h:
                    visible = False

if elbow_angle > 165:
    if stage == "down":
        stage = "up"
        counter += 1
    else:
        stage = "up"
elif elbow_angle < 95 and stage == "up":
    stage = "down"
"""

run1: 
code failed 145
code success 55

labelled correct: 100
code failed 73
code success 27

labelled incorrect: 100
code failed 72
code success 28


'\nif data_range > 0.5 * h:\n                    visible = False\n\nif elbow_angle > 165:\n    if stage == "down":\n        stage = "up"\n        counter += 1\n    else:\n        stage = "up"\nelif elbow_angle < 95 and stage == "up":\n    stage = "down"\n'

In [ ]:
print(f"run2: \ncode failed {107}\ncode success {93}\n\nlabelled correct: {100}\ncode failed {52}\ncode success {48}\n\nlabelled incorrect: {100}\ncode failed {55}\ncode success {45}")
"""
# NO HORIZONTAL FILTER/DATA RANGE CHECK

if elbow_angle > 130:
    if stage == "down":
        stage = "up"
        counter += 1
    else:
        stage = "up"
elif elbow_angle < 110 and stage == "up":
    stage = "down"
"""

run2: 
code failed 107
code success 93

labelled correct: 100
code failed 52
code success 48

labelled incorrect: 100
code failed 55
code success 45


'\n# NO HORIZONTAL FILTER/DATA RANGE CHECK\n\nif elbow_angle > 130:\n    if stage == "down":\n        stage = "up"\n        counter += 1\n    else:\n        stage = "up"\nelif elbow_angle < 110 and stage == "up":\n    stage = "down"\n'

# rep3 breakdown

In [ ]:
print(f"run3: \ncode failed {20}\ncode success {180}\n\nlabelled correct: {100}\ncode failed {7}\ncode success {93}\n\nlabelled incorrect: {100}\ncode failed {13}\ncode success {87}")
"""
# NO HORIZONTAL FILTER/DATA RANGE CHECK

if elbow_angle > 130:
    if stage == "down":
        stage = "up"
        counter += 1
    else:
        stage = "up"
elif elbow_angle < 110 and stage == "up":
    stage = "down"

# removed ankle visibility check

# CHANGED VISIBILITY METRIC TO EITHER RIGHT OR LEFT SIDE BASED ON WHICH IS BETTER
"""

run3: 
code failed 20
code success 180

labelled correct: 100
code failed 7
code success 93

labelled incorrect: 100
code failed 13
code success 87


'\n# NO HORIZONTAL FILTER/DATA RANGE CHECK\n\nif elbow_angle > 130:\n    if stage == "down":\n        stage = "up"\n        counter += 1\n    else:\n        stage = "up"\nelif elbow_angle < 110 and stage == "up":\n    stage = "down"\n\n# removed ankle visibility check\n\n# CHANGED VISIBILITY METRIC TO EITHER RIGHT OR LEFT SIDE BASED ON WHICH IS BETTER\n'

every failed vid and why (20 items):

1- the wrist from the side of the person facing the camera is not in frame

2- flipped of 1

3- same as 1    ***

4- same as 1    ***

5- ankles of person not in frame

6- flipped of 5

7- ankles of person not in frame    ***

8- ankles of person not in frame    ***

9- shoulder visibility low      ***

10- shoulder visibility low      ***

11- shoulder visibility low      ***

12- hip dip; elbow not low enough

13- flipped of 12

14- unsure; shoulder visibility?

15- flipped of 14

16- unsure; shoulder visibility?

17- flipped of 16

18- unsure; shoulder visibility?

19- flipped of 18

20- multiple reps? unsure       ***




*** - flipped of this not fail? == maybe the code to flip didnt flip it?

In [4]:
print(f"run4: \ncode failed {19}\ncode success {179}\n\nlabelled correct: {100}\ncode failed {8}\ncode success {92}\n\nlabelled incorrect: {100}\ncode failed {13}\ncode success {87}")
"""
# NO HORIZONTAL FILTER/DATA RANGE CHECK

if elbow_angle > 155:
    if stage == "down":
        stage = "up"
        counter += 1
    else:
        stage = "up"
elif elbow_angle < 110 and stage == "up":
    stage = "down"

# added ankle visibility check back

# CHANGED VISIBILITY METRIC TO EITHER RIGHT OR LEFT SIDE BASED ON WHICH IS BETTER
"""

run4: 
code failed 19
code success 179

labelled correct: 100
code failed 8
code success 92

labelled incorrect: 100
code failed 13
code success 87


'\n# NO HORIZONTAL FILTER/DATA RANGE CHECK\n\nif elbow_angle > 155:\n    if stage == "down":\n        stage = "up"\n        counter += 1\n    else:\n        stage = "up"\nelif elbow_angle < 110 and stage == "up":\n    stage = "down"\n\n# added ankle visibility check back\n\n# CHANGED VISIBILITY METRIC TO EITHER RIGHT OR LEFT SIDE BASED ON WHICH IS BETTER\n'

todo:

look through what code missed with group and make central decisions on what to keep

clean up code of emoji things

clean up unneccessary imports

apply changes to model accurately and run with PushupKeyPoints to get 30 frame splices for model and train

Repurpose model to look at all 30 frames properly

fix and fine tune live M1